In [5]:
import os
import sys
import textwrap
from pathlib import Path

# --- CONFIGURATION ---
PROJECT_NAME = "library-inventory-manager-Sagar_Pawar"
BASE_DIR = Path.cwd() / PROJECT_NAME

# --- ROBUST CONTENT GENERATION ---
# We use textwrap.dedent to remove any accidental indentation from copy-pasting

def get_book_code():
    return textwrap.dedent("""
        class Book:
            def __init__(self, title, author, isbn, status='available'):
                self.title = title
                self.author = author
                self.isbn = isbn
                self.status = status

            def issue(self):
                if self.status == 'available':
                    self.status = 'issued'
                    return True
                return False

            def return_book(self):
                self.status = 'available'

            def to_dict(self):
                return {
                    "title": self.title,
                    "author": self.author,
                    "isbn": self.isbn,
                    "status": self.status
                }

            def __str__(self):
                return f"'{self.title}' by {self.author} (ISBN: {self.isbn}) - [{self.status}]"
    """).strip()

def get_inventory_code():
    return textwrap.dedent("""
        import json
        import logging
        from pathlib import Path
        try:
            from .book import Book
        except ImportError:
            from book import Book

        class LibraryInventory:
            def __init__(self, file_path=None):
                # Determine path relative to this file
                if file_path is None:
                    # Assumes structure: root/library_manager/inventory.py
                    base_path = Path(__file__).resolve().parent.parent
                    self.file_path = base_path / 'data' / 'catalog.json'
                else:
                    self.file_path = Path(file_path)
                    
                self.books = []
                self.load_books()

            def add_book(self, title, author, isbn):
                new_book = Book(title, author, isbn)
                self.books.append(new_book)
                self.save_books()
                logging.info(f"Book added: {title}")

            def save_books(self):
                data = [book.to_dict() for book in self.books]
                try:
                    # Ensure directory exists before saving
                    if not self.file_path.parent.exists():
                        self.file_path.parent.mkdir(parents=True, exist_ok=True)
                        
                    with open(self.file_path, 'w') as f:
                        json.dump(data, f, indent=4)
                except IOError as e:
                    logging.error(f"Failed to save data: {e}")

            def load_books(self):
                if not self.file_path.exists():
                    return 
                try:
                    with open(self.file_path, 'r') as f:
                        content = f.read()
                        if not content: return
                        data = json.loads(content)
                        self.books = [Book(**item) for item in data]
                except (json.JSONDecodeError, IOError):
                    logging.error("Data file corrupted or unreadable.")
                    self.books = []

            def search_by_isbn(self, isbn):
                for book in self.books:
                    if book.isbn == isbn:
                        return book
                return None

            def search_by_title(self, title):
                results = []
                for book in self.books:
                    if title.lower() in book.title.lower():
                        results.append(book)
                return results

            def display_all(self):
                if not self.books:
                    print("No books in inventory.")
                for book in self.books:
                    print(book)
    """).strip()

def get_main_code():
    # Note: Double backslashes \\n are needed so they write as \n in the file
    return textwrap.dedent("""
        import sys
        import os
        import logging

        # 1. Add project root to path so we can import library_manager
        current_dir = os.path.dirname(os.path.abspath(__file__))
        project_root = os.path.abspath(os.path.join(current_dir, '..'))
        if project_root not in sys.path:
            sys.path.insert(0, project_root)

        try:
            from library_manager.inventory import LibraryInventory
        except ImportError as e:
            print(f"CRITICAL ERROR: Could not import library modules. {e}")
            print(f"Checked path: {project_root}")
            sys.exit(1)

        # Logging Setup
        logging.basicConfig(
            filename='library.log',
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )

        def print_menu():
            print("\\n" + "="*30)
            print(" LIBRARY INVENTORY MANAGER")
            print("="*30)
            print("1. Add New Book")
            print("2. Issue Book")
            print("3. Return Book")
            print("4. Search for Book")
            print("5. View All Books")
            print("6. Exit")
            print("="*30)

        def main():
            library = LibraryInventory()

            while True:
                print_menu()
                choice = input("Enter your choice (1-6): ").strip()

                try:
                    if choice == '1':
                        print("\\n--- Add New Book ---")
                        title = input("Enter Title: ").strip()
                        author = input("Enter Author: ").strip()
                        isbn = input("Enter ISBN: ").strip()
                        if title and author and isbn:
                            library.add_book(title, author, isbn)
                            print(f"Success: '{title}' added.")
                        else:
                            print("Error: All fields are required.")

                    elif choice == '2':
                        isbn = input("Enter ISBN to issue: ").strip()
                        book = library.search_by_isbn(isbn)
                        if book:
                            if book.issue():
                                library.save_books()
                                print(f"Success: '{book.title}' issued.")
                            else:
                                print(f"Warning: '{book.title}' is already issued.")
                        else:
                            print("Error: Book not found.")

                    elif choice == '3':
                        isbn = input("Enter ISBN to return: ").strip()
                        book = library.search_by_isbn(isbn)
                        if book:
                            book.return_book()
                            library.save_books()
                            print(f"Success: '{book.title}' returned.")
                        else:
                            print("Error: Book not found.")

                    elif choice == '4':
                        term = input("Enter Title or ISBN: ").strip()
                        found = library.search_by_title(term)
                        if not found:
                            res = library.search_by_isbn(term)
                            if res: found = [res]
                        
                        if found:
                            print(f"\\nFound {len(found)} matches:")
                            for b in found: print(b)
                        else:
                            print("No matches found.")

                    elif choice == '5':
                        print("\\n--- Current Catalog ---")
                        library.display_all()

                    elif choice == '6':
                        print("Goodbye!")
                        break
                    else:
                        print("Invalid choice.")

                except Exception as e:
                    logging.error(f"Runtime Error: {e}")
                    print(f"An error occurred: {e}")

        if __name__ == "__main__":
            main()
    """).strip()

def get_test_code():
    return textwrap.dedent("""
        import unittest
        import sys
        import os

        # Add project root to path
        sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

        from library_manager.book import Book

        class TestBook(unittest.TestCase):
            def test_initialization(self):
                b = Book("Python 101", "Guido", "12345")
                self.assertEqual(b.title, "Python 101")
                self.assertEqual(b.status, "available")

            def test_issue_return(self):
                b = Book("Test Book", "Me", "000")
                b.issue()
                self.assertEqual(b.status, "issued")
                b.return_book()
                self.assertEqual(b.status, "available")

        if __name__ == '__main__':
            unittest.main()
    """).strip()

# --- FILE WRITING LOGIC ---

def create_project():
    print(f"Fixing/Creating project at: {BASE_DIR}")
    
    if not BASE_DIR.exists():
        BASE_DIR.mkdir()

    # Define file structure
    files = {
        "library_manager/__init__.py": "",
        "library_manager/book.py": get_book_code(),
        "library_manager/inventory.py": get_inventory_code(),
        "cli/__init__.py": "",
        "cli/main.py": get_main_code(),
        "data/catalog.json": "[]",
        "tests/__init__.py": "",
        "tests/test_book.py": get_test_code(),
        "README.md": "# Library Project\n\nRun with: `python cli/main.py`",
        "requirements.txt": "pytest"
    }

    for rel_path, content in files.items():
        file_path = BASE_DIR / rel_path
        
        # Create folder if missing
        if not file_path.parent.exists():
            file_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Write file
        try:
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)
            print(f"✔ Generated: {rel_path}")
        except Exception as e:
            print(f"❌ Failed to write {rel_path}: {e}")

    print("\n" + "="*40)
    print("PROJECT REPAIRED SUCCESSFULLY")
    print("="*40)
    print("Try running the following commands:")
    print(f"1. cd {PROJECT_NAME}")
    print("2. python cli/main.py")

if __name__ == "__main__":
    create_project()

Fixing/Creating project at: C:\Users\hp\Downloads\python assignments\Library_inventory\library-inventory-manager-Sagar_Pawar
✔ Generated: library_manager/__init__.py
✔ Generated: library_manager/book.py
✔ Generated: library_manager/inventory.py
✔ Generated: cli/__init__.py
✔ Generated: cli/main.py
✔ Generated: data/catalog.json
✔ Generated: tests/__init__.py
✔ Generated: tests/test_book.py
✔ Generated: README.md
✔ Generated: requirements.txt

PROJECT REPAIRED SUCCESSFULLY
Try running the following commands:
1. cd library-inventory-manager-Sagar_Pawar
2. python cli/main.py


In [7]:
python cli/main.py

SyntaxError: invalid syntax (2517879977.py, line 1)